## TF Functions and Concrete Functions

TF Functions are polymorphic, meaning they support inputs of different types (and shapes). For example, consider the following `tf_cube()` function:

In [1]:
import tensorflow as tf
@tf.function
def tf_cube(x):
    return x ** 3

In [2]:
tf_cube(3)

<tf.Tensor: shape=(), dtype=int32, numpy=27>

In [5]:
tf_cube(tf.constant(3.0))

<tf.Tensor: shape=(), dtype=float32, numpy=27.0>

In [4]:
tf_cube(tf.constant([2.0]))

<tf.Tensor: shape=(1,), dtype=float32, numpy=array([8.], dtype=float32)>

In [3]:
tf_cube(tf.constant([[1.0, 2.0], [3.0, 4.0]]))

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[ 1.,  8.],
       [27., 64.]], dtype=float32)>

In [6]:
concrete_function = tf_cube.get_concrete_function(tf.constant(2.0))
concrete_function

<ConcreteFunction tf_cube(x) at 0x130224EFEE0>

In [7]:
concrete_function(tf.constant(2.0))

<tf.Tensor: shape=(), dtype=float32, numpy=8.0>

Figure G-1 shows the `tf_cube()` TF Function, after we called `tf_cube(2)` and `tf_cube(tf.constant(2.0))`: two *concrete functions* were generated, one for each signature, each with its own optimized function graph (`FuncGraph`), and its own function definition (`FunctionDef`). A function definition points to the parts of the graph that correspond to the function’s inputs and outputs. In each `FuncGraph`, the nodes (ovals) represent operations (e.g., power, constants, or placeholders for arguments like `x`), while the edges (the solid arrows between the operations) represent the tensors that will flow through the graph. The *concrete function* on the left is specialized for `x = 2`, so TensorFlow managed to simplify it to just output `8` all the time (note that the function definition does not even have an input). The *concrete function* on the right is specialized for float32 scalar tensors, and it could not be simplified. If we call `tf_cube(tf.constant(5.0))`, the second concrete function will be called, the placeholder operation for `x` will output `5.0`, then the power operation will compute `5.0 ** 3`, so the output will be `125.0`. 

<img src="./chapters/G/1.png">
<div style="text-align:center"> Figure G-1. The tf_cube() TF Function, with its ConcreteFunctions and their FunctionGraphs </div>  

Now let’s continue to peek under the hood, and see how to access function definitions and function graphs and how to explore a graph’s operations and tensors.

## Exploring Function Definitions and Graphs  

You can access a concrete function’s computation graph using the `graph` attribute, and get the list of its operations by calling the graph’s `get_operations()` method:

In [1]:
import tensorflow as tf
@tf.function
def tf_cube(x):
    return x ** 3

In [10]:
concrete_function = tf_cube.get_concrete_function(tf.constant(2.0))
concrete_function(tf.constant(2.0))

<tf.Tensor: shape=(), dtype=float32, numpy=8.0>

In [8]:
concrete_function.graph

In [9]:
ops = concrete_function.graph.get_operations()
ops

[<tf.Operation 'x' type=Placeholder>,
 <tf.Operation 'pow/y' type=Const>,
 <tf.Operation 'pow' type=Pow>,
 <tf.Operation 'Identity' type=Identity>]

In this example, the first operation represents the input argument `x` (it is called a *placeholder*), the second “operation” represents the constant `3`, the third operation represents the power operation (`**`), and the final operation represents the output of this function (it is an identity operation, meaning it will do nothing more than copy the output of the addition operation). Each operation has a list of input and output tensors that you can easily access using the operation’s `inputs` and `outputs` attributes. For example, let’s get the list of inputs and outputs of the power operation:

In [12]:
ops[2]

<tf.Operation 'pow' type=Pow>

In [11]:
pow_op = ops[2]
list(pow_op.inputs)

[<tf.Tensor 'x:0' shape=() dtype=float32>,
 <tf.Tensor 'pow/y:0' shape=() dtype=float32>]

In [13]:
pow_op.outputs

[<tf.Tensor 'pow:0' shape=() dtype=float32>]

This computation graph is represented in Figure G-2.  

<img src="./chapters/G/2.png">
<div style="text-align:center"> Figure G-2. Example of a computation graph </div>  

Note that each operation has a name. It defaults to the name of the operation (e.g., "`pow`"), but you can define it manually when calling the operation (e.g., `tf.pow(x, 3, name="other_name")`). If a name already exists, TensorFlow automatically adds a unique index (e.g., "`pow_1`", "`pow_2`", etc.). Each tensor also has a unique name: it is always the name of the operation that outputs this tensor, plus `:0` if it is the operation’s first output, or `:1` if it is the second output, and so on. You can fetch an operation or a tensor by name using the graph’s `get_operation_by_name()` or `get_tensor_by_name()` methods:

In [14]:
concrete_function.graph.get_operation_by_name('x')

<tf.Operation 'x' type=Placeholder>

In [15]:
concrete_function.graph.get_tensor_by_name('Identity:0')

<tf.Tensor 'Identity:0' shape=() dtype=float32>

The concrete function also contains the function definition (represented as a protocol buffer), which includes the function’s signature. This signature allows the concrete function to know which placeholders to feed with the input values, and which tensors to return:

In [16]:
concrete_function.function_def.signature

name: "__inference_tf_cube_29"
input_arg {
  name: "x"
  type: DT_FLOAT
}
output_arg {
  name: "identity"
  type: DT_FLOAT
}

## A Closer Look at Tracing  

Let’s tweak the `tf_cube()` function to print its input:

In [17]:
@tf.function
def tf_cube(x):
    print("x =", x)
    return x ** 3

In [18]:
result = tf_cube(tf.constant(2.0))

x = Tensor("x:0", shape=(), dtype=float32)


In [19]:
result

<tf.Tensor: shape=(), dtype=float32, numpy=8.0>

The `result` looks good, but look at what was printed: `x` is a symbolic tensor! It has a shape and a data type, but no value. Plus it has a name ("`x:0`"). This is because the `print()` function is not a TensorFlow operation, so it will only run when the Python function is traced, which happens in graph mode, with arguments replaced with symbolic tensors (same type and shape, but no value). Since the `print()` function was not captured into the graph, the next times we call `tf_cube()` with float32 scalar tensors, nothing is printed:

In [20]:
result = tf_cube(tf.constant(3.0))

In [21]:
result = tf_cube(tf.constant(4.0))

In [22]:
result

<tf.Tensor: shape=(), dtype=float32, numpy=64.0>

But if we call `tf_cube()` with a tensor of a different type or shape, or with a new Python value, the function will be traced again, so the `print()` function will be called:

In [23]:
result = tf_cube(2) # new Python value: trace!

x = 2


In [24]:
result = tf_cube(3) # new Python value: trace!

x = 3


In [25]:
result = tf_cube(tf.constant([[1., 2.]])) # New shape: trace!

x = Tensor("x:0", shape=(1, 2), dtype=float32)


In [26]:
result = tf_cube(tf.constant([[3., 4.], [5., 6.]])) # New shape: trace!

x = Tensor("x:0", shape=(2, 2), dtype=float32)


In [27]:
result = tf_cube(tf.constant([[7., 8.], [9., 10.]])) # Same shape: no trace

In [28]:
result = tf_cube(tf.constant(4.0))

In [29]:
result = tf_cube(3) # new Python value: trace!

In [30]:
result = tf_cube(4) # new Python value: trace!

x = 4


In some cases, you may want to restrict a TF Function to a specific input signature. For example, suppose you know that you will only ever call a TF Function with batches of 28 × 28–pixel images, but the batches will have very different sizes. You may not want TensorFlow to generate a different concrete function for each batch size, or count on it to figure out on its own when to use `None`. In this case, you can specify the input signature like this:

In [31]:
@tf.function(input_signature=[tf.TensorSpec([None, 28, 28], tf.float32)])
def shrink(images):
    return images[:, ::2, ::2] # drop half the rows and columns

This TF Function will accept any float32 tensor of shape `[*, 28, 28]`, and it will reuse the same concrete function every time:

In [32]:
img_batch_1 = tf.random.uniform(shape=[100, 28, 28])
img_batch_2 = tf.random.uniform(shape=[50, 28, 28])
preprocessed_images = shrink(img_batch_1) # Works fine. Traces the function.

In [33]:
preprocessed_images = shrink(img_batch_2) # Works fine. Same concrete function.

In [37]:
preprocessed_images.shape

TensorShape([50, 14, 14])

However, if you try to call this TF Function with a Python value, or a tensor of an unexpected data type or shape, you will get an exception:

In [38]:
img_batch_3 = tf.random.uniform(shape=[2, 2, 2])
preprocessed_images = shrink(img_batch_3)  # ValueError! Unexpected signature.

ValueError: Python inputs incompatible with input_signature:
  inputs: (
    tf.Tensor(
[[[0.21487963 0.8821975 ]
  [0.9357896  0.19632602]]

 [[0.24513781 0.18792224]
  [0.5411782  0.5384644 ]]], shape=(2, 2, 2), dtype=float32))
  input_signature: (
    TensorSpec(shape=(None, 28, 28), dtype=tf.float32, name=None)).

## Using AutoGraph to Capture Control Flow  

If your function contains a simple `for` loop, what do you expect will happen? For example, let’s write a function that will add `10` to its input, by just adding 1 10 times:

In [39]:
@tf.function
def add_10(x):
    for i in range(10):
        x += 1
    return x

It works fine, but when we look at its graph, we find that it does not contain a loop: it just contains 10 addition operations!

In [40]:
add_10(tf.constant(0))

<tf.Tensor: shape=(), dtype=int32, numpy=10>

In [41]:
add_10(tf.constant(0.0))

<tf.Tensor: shape=(), dtype=float32, numpy=10.0>

In [42]:
add_10(0)

<tf.Tensor: shape=(), dtype=int32, numpy=10>

In [43]:
add_10.get_concrete_function(tf.constant(0)).graph.get_operations()

[<tf.Operation 'x' type=Placeholder>,
 <tf.Operation 'add/y' type=Const>,
 <tf.Operation 'add' type=AddV2>,
 <tf.Operation 'add_1/y' type=Const>,
 <tf.Operation 'add_1' type=AddV2>,
 <tf.Operation 'add_2/y' type=Const>,
 <tf.Operation 'add_2' type=AddV2>,
 <tf.Operation 'add_3/y' type=Const>,
 <tf.Operation 'add_3' type=AddV2>,
 <tf.Operation 'add_4/y' type=Const>,
 <tf.Operation 'add_4' type=AddV2>,
 <tf.Operation 'add_5/y' type=Const>,
 <tf.Operation 'add_5' type=AddV2>,
 <tf.Operation 'add_6/y' type=Const>,
 <tf.Operation 'add_6' type=AddV2>,
 <tf.Operation 'add_7/y' type=Const>,
 <tf.Operation 'add_7' type=AddV2>,
 <tf.Operation 'add_8/y' type=Const>,
 <tf.Operation 'add_8' type=AddV2>,
 <tf.Operation 'add_9/y' type=Const>,
 <tf.Operation 'add_9' type=AddV2>,
 <tf.Operation 'Identity' type=Identity>]

This actually makes sense: when the function got traced, the loop ran 10 times, so the `x += 1` operation was run 10 times, and since it was in graph mode, it recorded this operation 10 times in the graph. You can think of this `for` loop as a “static” loop that gets unrolled when the graph is created.

If you want the graph to contain a “dynamic” loop instead (i.e., one that runs when the graph is executed), you can create one manually using the `tf.while_loop()` operation, but it is not very intuitive (see the “Using AutoGraph to Capture Control Flow” section of the Chapter 12 notebook for an example). Instead, it is much simpler to use TensorFlow’s *AutoGraph* feature, discussed in Chapter 12. 

`AutoGraph` is actually activated by default (if you ever need to turn it off, you can pass `autograph=False` to `tf.function()`). So if it is on, why didn’t it capture the for loop in the `add_10()` function? Well, it only captures for loops that iterate over `tf.range()`, not `range()`. This is to give you the choice:

  - If you use `range()`, the for loop will be static, meaning it will only be executed when the function is traced. The loop will be “unrolled” into a set of operations for each iteration, as we saw.

  - If you use `tf.range()`, the loop will be dynamic, meaning that it will be included in the graph itself (but it will not run during tracing).

Let’s look at the graph that gets generated if you just replace `range()` with `tf.range()` in the `add_10()` function:

In [45]:
@tf.function
def add_10(x):
    for i in tf.range(10):
        x += 1
    return x

In [47]:
add_10(0)

<tf.Tensor: shape=(), dtype=int32, numpy=10>

In [48]:
add_10.get_concrete_function(tf.constant(0)).graph.get_operations()

[<tf.Operation 'x' type=Placeholder>,
 <tf.Operation 'range/start' type=Const>,
 <tf.Operation 'range/limit' type=Const>,
 <tf.Operation 'range/delta' type=Const>,
 <tf.Operation 'range' type=Range>,
 <tf.Operation 'sub' type=Sub>,
 <tf.Operation 'floordiv' type=FloorDiv>,
 <tf.Operation 'mod' type=FloorMod>,
 <tf.Operation 'zeros_like' type=Const>,
 <tf.Operation 'NotEqual' type=NotEqual>,
 <tf.Operation 'Cast' type=Cast>,
 <tf.Operation 'add' type=AddV2>,
 <tf.Operation 'zeros_like_1' type=Const>,
 <tf.Operation 'Maximum' type=Maximum>,
 <tf.Operation 'while/maximum_iterations' type=Const>,
 <tf.Operation 'while/loop_counter' type=Const>,
 <tf.Operation 'while' type=StatelessWhile>,
 <tf.Operation 'Identity' type=Identity>]

As you can see, the graph now contains a `While` loop operation, as if you had called the `tf.while_loop()` function.

## Handling Variables and Other Resources in TF Functions  

In TensorFlow, variables and other stateful objects, such as queues or datasets, are called *resources*. TF Functions treat them with special care: any operation that reads or updates a resource is considered stateful, and TF Functions ensure that stateful operations are executed in the order they appear (as opposed to stateless operations, which may be run in parallel, so their order of execution is not guaranteed). Moreover, when you pass a resource as an argument to a TF Function, it gets passed by reference, so the function may modify it. For example:

In [49]:
counter = tf.Variable(0)

@tf.function
def increment(counter, c=1):
    return counter.assign_add(c)

increment(counter) # counter is now equal to 1

<tf.Tensor: shape=(), dtype=int32, numpy=1>

In [50]:
increment(counter) # counter is now equal to 2

<tf.Tensor: shape=(), dtype=int32, numpy=2>

If you peek at the function definition, the first argument is marked as a resource:

In [56]:
function_def = increment.get_concrete_function(counter).function_def

In [52]:
function_def.signature.input_arg[0]

name: "counter"
type: DT_RESOURCE

It is also possible to use a `tf.Variable` defined outside of the function, without explicitly passing it as an argument:

In [53]:
counter = tf.Variable(0)

@tf.function
def increment(c=1):
    return counter.assign_add(c)

In [54]:
increment()

<tf.Tensor: shape=(), dtype=int32, numpy=1>

In [55]:
increment()

<tf.Tensor: shape=(), dtype=int32, numpy=2>

In [57]:
function_def = increment.get_concrete_function(counter).function_def

In [58]:
function_def.signature.input_arg[0]

name: "c"
type: DT_RESOURCE

The TF Function will treat this as an implicit first argument, so it will actually end up with the same signature (except for the name of the argument). However, using global variables can quickly become messy, so you should generally wrap variables (and other resources) inside classes. The good news is `@tf.function` works fine with methods too:

In [59]:
class Counter:
    def __init__(self):
        self.counter = tf.Variable(0)

    @tf.function
    def increment(self, c=1):
        return self.counter.assign_add(c)

##  Using TF Functions with tf.keras (or Not)  

By default, any custom function, layer, or model you use with `tf.keras` will automatically be converted to a TF Function; you do not need to do anything at all! However, in some cases you may want to deactivate this automatic conversion—for example, if your custom code cannot be turned into a TF Function, or if you just want to debug your code, which is much easier in eager mode. To do this, you can simply pass `dynamic=True` when creating the model or any of its layers:

In [ ]:
model = MyModel(dynamic=True)

If your custom model or layer will always be dynamic, you can instead call the base class’s constructor with `dynamic=True`:

In [ ]:
class MyLayer(keras.layers.Layer):
    def __init__(self, units, **kwargs):
        super().__init__(dynamic=True, **kwargs)
        [...]

Alternatively, you can pass `run_eagerly=True` when calling the `compile()` method:

In [ ]:
model.compile(loss=my_mse, optimizer="nadam", metrics=[my_mae],
              run_eagerly=True)

Now you know how TF Functions handle polymorphism (with multiple concrete functions), how graphs are automatically generated using AutoGraph and tracing, what graphs look like, how to explore their symbolic operations and tensors, how to handle variables and resources, and how to use TF Functions with `tf.keras`.